# Stage 3 – Top2Vec (GPU notebook)

> מומלץ להפעיל GPU ב-Kaggle: *Settings → Accelerator → GPU*.

המחברת הזו מריצה רק Top2Vec ושומרת מודלים ל-`MODELS_DIR/top2vec_gpu`.


## 0. חיבור ל-Google Drive והגדרת נתיבים

אם אתה בקולאב – הרץ את התא הזה. אם לוקאלי, אפשר להתאים את `BASE_DIR`.

In [1]:
import sys
from pathlib import Path

def _add_repo_root():
    cwd = Path.cwd()
    for path in [cwd] + list(cwd.parents):
        if (path / 'project_paths.py').exists():
            sys.path.insert(0, str(path))
            return path
    return cwd

_add_repo_root()

try:
    from google.colab import drive
    drive.mount('/content/drive')
except Exception:
    pass

from project_paths import get_paths

paths = get_paths().ensure()
BASE_DIR = paths.root
DATA_DIR = paths.data_dir
MODELS_DIR = paths.models_dir
RESULTS_DIR = paths.results_dir
RESULTS_ROOT = RESULTS_DIR

print('BASE_DIR  :', BASE_DIR)
print('DATA_DIR  :', DATA_DIR)
print('MODELS_DIR:', MODELS_DIR)
print('RESULTS_DIR:', RESULTS_DIR)


Mounted at /content/drive
BASE_DIR  : /content/drive/MyDrive/HebrewTopicModel
DATA_DIR  : /content/drive/MyDrive/HebrewTopicModel/data
MODELS_DIR: /content/drive/MyDrive/HebrewTopicModel/models


In [2]:
# =========================
# Kaggle compatibility helper (optional)
# - Copies required parquet files from /kaggle/input into DATA_DIR if missing
# - Initializes best_lda_info to avoid NameError later
# =========================

import shutil
from pathlib import Path

best_lda_info = None  # will be populated later (after LDA runs)

def _is_kaggle() -> bool:
    return Path("/kaggle").exists()

required_files = ["processed_wiki.parquet", "processed_ugc.parquet"]

if _is_kaggle():
    input_root = Path("/kaggle/input")
    print("🟣 Kaggle detected. Searching for required files under:", input_root)

    for fname in required_files:
        target = DATA_DIR / fname
        if target.exists():
            print("✅ already in DATA_DIR:", target)
            continue

        matches = list(input_root.rglob(fname))
        if not matches:
            print(f"⚠️ Could not find {fname} under /kaggle/input. "
                  f"Make sure your Kaggle Dataset contains it.")
            continue

        src = matches[0]
        target.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy2(src, target)
        print(f"✅ copied {fname} ->", target)

else:
    print("ℹ️ Not running on Kaggle (or /kaggle not found). Skipping Kaggle sync.")


🟣 Kaggle detected. Searching for required files under: /kaggle/input
✅ already in DATA_DIR: /content/drive/MyDrive/HebrewTopicModel/data/processed_wiki.parquet
✅ already in DATA_DIR: /content/drive/MyDrive/HebrewTopicModel/data/processed_ugc.parquet


In [3]:
import warnings
warnings.filterwarnings(action="ignore", message=r"datetime.datetime.utcnow")
warnings.filterwarnings(
    "ignore",
    category=DeprecationWarning,
    message=r".*datetime\.datetime\.utcnow\(\) is deprecated.*"
)

warnings.filterwarnings("ignore", category=DeprecationWarning, module="multiprocessing")

import os
os.environ['PYTHONWARNINGS'] = 'ignore::DeprecationWarning'

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)



## 1. התקנת ספריות וייבוא

נשתמש ב־pandas, sklearn, gensim ו-Top2Vec.

In [4]:
!pip install -q pandas tqdm gensim scikit-learn top2vec seaborn matplotlib pyldavis

import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
import joblib

from top2vec import Top2Vec

import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

import gensim
from gensim import corpora
from gensim.models import CoherenceModel, LdaMulticore, LsiModel, TfidfModel, Phrases
from gensim.models.phrases import Phraser

# sklearn לנMF
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 119.5 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/hdbscan/robust_single_linkage_.py:175: SyntaxWarning: invalid escape sequence '\{'
  $max \{ core_k(a), core_k(b), 1/\alpha d(a,b) \}$.


## Load data
הטעינה מתבצעת מתוך `DATA_DIR` וממלאת NaN כדי למנוע קריסות בפיצול טוקנים.

נדרש: `processed_wiki.parquet` ו־`processed_ugc.parquet` תחת `DATA_DIR`.


In [5]:
print("⏳ טוען נתונים...")
datasets = {}
for name, p in [('Wiki', "processed_wiki.parquet"), ('UGC', "processed_ugc.parquet")]:
    path = DATA_DIR / p
    if path.exists():
        df = pd.read_parquet(path)
        # מילוי NaN למניעת קריסה ב-split
        df['clean_text'] = df['clean_text'].fillna("")
        df['lemma_text'] = df['lemma_text'].fillna("")
        datasets[name] = df
        print(f"✅ {name} נטען ({len(df)} שורות)")

assert len(datasets) > 0, "לא נטען אף דאטהסט — בדוק נתיבים/קבצים."

wiki_count = len(datasets.get('Wiki', [])) # Use .get to safely handle cases where 'Wiki' might not be present
ugc_count = len(datasets.get('UGC', [])) # Use .get to safely handle cases where 'UGC' might not be present
print(f"✅ נטענו בהצלחה: {wiki_count} ערכי ויקיפדיה, {ugc_count} תגובות UGC.")

⏳ טוען נתונים...
✅ Wiki נטען (24999 שורות)
✅ UGC נטען (5098 שורות)
✅ נטענו בהצלחה: 24999 ערכי ויקיפדיה, 5098 תגובות UGC.


In [6]:
datasets["UGC"].head()

,text,title,emotion,sentiment,sen_score,source,clean_text,lemma_text
0,גם בסופרפארם בתור בלי י מסכה.למה?&quot;לא בא...,"""אובדן מוחלט של אמון הציבור בהנהגה"",",הבעת אמון,שלילי,-0.027778,ynet,גם בסופרפארם בתור בלי י מסכה.למה? לא בא לי.תשא...,תור בל מסכה למה בא לי נשאר בית תשובה אישה נדחף...
1,האשם הוא ביבי ששפך על החרדים והמתנחלים מיליאר...,"""הערכות באוצר: עלות הבידוד למשק תגיע ל-4.3 מיל...",הבעת אמון,שלילי,-0.333333,ynet,האשם הוא ביבי ששפך על החרדים והמתנחלים מיליארד...,אשם ביבי חרד מיליארד כיסא אפשר פושע חשוד מולדת...
2,זה מה שקורה שביבי עסוק בשוויוק עצמו במקום בנ...,"""ישראל הרבה מאחורי העולם המערבי במספר החוקרים ...",הבעת אמון,שלילי,-0.122222,ynet,זה מה שקורה שביבי עסוק בשוויוק עצמו במקום בניה...,קורה ביבי עסוק ספין עצם ניהול הוכחה ביב דיבור ...
3,מלכתחילה זה שולם ע״י אזרחי ישראל .... זה חוב ...,"""50% מהמענקים כבר הועברו, היום מתחילה החלוקה ל...",הבעת אמון,שלילי מאוד,0.246914,ynet,מלכתחילה זה שולם ע״י אזרחי ישראל .... זה חוב ש...,מלכתחילה שולם אזרח ישראל חוב ילד שילם
4,משפ.נתניהו כמה אוהבים אותכם...ובכל הארץ...עו...,"""אלפים מוחים בת""א, מפגינים חסמו כביש: ""הממשלה ...",הבעת אמון,חיובי,0.555556,ynet,משפ.נתניהו כמה אוהבים אותכם...ובכל הארץ...עופו...,נתניהו אוהב ארץ ארץ צומת ישראל אמיתי עובד שירת...


In [7]:
datasets["Wiki"].head()

,text,source,dominant_emotion,clean_text,lemma_text
0,באלגברה ליניארית באוניברסיטת הלסינקי פותרות תר...,wikipedia,wiki_fact,באלגברה ליניארית באוניברסיטת הלסינקי פותרות תר...,אלגברה לינארי אוניברסיטה הלסינקי פותר תרגיל חש...
1,"אַקְסיּוֹמָה, אמיתה, או הנחת יסודאוצר מילים עב...",wikipedia,wiki_fact,"אַקְסיּוֹמָה, אמיתה, או הנחת יסודאוצר מילים עב...",אקסיומה הנחה יסוד מילה עברי מונח אקסיומה קשור ...
2,240||דוגמאות שונות לתופעות פיזיקליות פִיזִיקָה...,wikipedia,wiki_fact,דוגמאות שונות לתופעות פיזיקליות פִיזִיקָה מהמי...,דוגמה שונה תופעה פיזיקלי פיזיקה מילה יווני טבע...
3,"ויקי (מהוואית: Wiki, ""מהיר"") היא שיטה לבניית מ...",wikipedia,wiki_fact,"ויקי מהוואית , מהיר היא שיטה לבניית מאגרי מידע...",ויקי מהיר שיטה בנייה מאגר מידע אתר אינטרנט שבו...
4,במופע מוזיקה בשנת 1964 מוזיקה (מיוונית עתיקה: ...,wikipedia,wiki_fact,במופע מוזיקה בשנת מוזיקה מיוונית עתיקה היא אמנ...,מופע מוזיקה שנה מוזיקה יווני עתיק אומנות סידור...


## GPU check
מוודא שטורץ' רואה CUDA (אם אין — Top2Vec ירוץ על CPU ויכול לקחת הרבה זמן).


In [8]:
import torch
print('torch:', torch.__version__)
print('cuda available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('cuda device:', torch.cuda.get_device_name(0))


torch: 2.9.0+cu126
cuda available: True
cuda device: Tesla T4


### Top2Vec – Embeddings‑based Topic Modeling

Top2Vec עובד במרחב Embeddings (מסמכים), מזהה אשכולות צפופים ומפיק מהם מילים מייצגות.

- לרוב מתקבלות תמות “סמנטיות” יותר (לא בהכרח BoW).
- יכול להיות רגיש לטקסט קצר מאוד/מילים פונקציונליות — לכן שווה לשים לב לתוצאות ב‑UGC.


### 6. מודל Top2Vec – Topic Modeling מבוסס Embeddings

Top2Vec:

- ממפה מסמכים למרחב Embedding (באמצעות מודל שפה).  
- מאתר אזורים צפופים במרחב כדי לזהות נושאים.  
- עבור כל נושא, מחזיר את המילים הייצוגיות ביותר.

הפונקציה הבאה:

- מאמנת מודל Top2Vec על המסמכים.  
- מחזירה:
  - `model` – המודל המאומן (לשמירה בהמשך).  
  - `topics` – רשימת נושאים, כל נושא 10 מילים.

> **הערה:** הרצת Top2Vec על full corpus עשויה לקחת זמן.  
> ניתן להתחיל עם `USE_SAMPLE = True` להרצה מהירה יותר.

המודלים נשמרים תחת `MODELS_DIR/top2vec_comparison` והארטיפקטים ב־`results_artifacts/`.
לזמן ריצה סביר מומלץ GPU; על CPU הריצה איטית מאוד.


**Note:**


Top2Vec, even with multilingual USE embeddings and deep-learn mode, tends to cluster Hebrew short-form UGC around discourse markers (e.g., ‘ש-’, ‘צריך’) rather than topical content. This highlights the importance of Hebrew-specific normalization and the limitations of generic embedding-based topic models on short noisy Hebrew data.

In [9]:
# Ensure sentence-transformers is available (needed by Top2Vec cell)
try:
    import sentence_transformers  # noqa: F401
except Exception as e:
    print("Installing sentence-transformers ...")
    !pip install -q sentence-transformers


In [10]:
import os, json, pickle, numpy as np

MODEL_NAMES = ["Top2Vec", "LDA", "LSA", "NMF"]
TEXT_TYPES  = ["clean_text", "lemma_text"]

configs = {
    'Wiki': {'ks': [10, 20, 30], 'no_above': 0.5},
    'UGC':  {'ks': [6, 10, 14], 'no_above': 0.7}
}

def init_best_store(datasets_dict, model_names, text_types):
    store = {}
    for ds_name in datasets_dict.keys():
        for m in model_names:
            for ttype in text_types:
                store[(ds_name, m, ttype)] = {"score": -np.inf, "payload": None}
    return store

best_store = init_best_store(datasets, MODEL_NAMES, TEXT_TYPES)

def consider_best(ds_name: str, model_type: str, txt_type: str, score: float, payload: dict):
    key = (ds_name, model_type, txt_type)
    if score is None or (isinstance(score, float) and np.isnan(score)):
        return
    if score > best_store[key]["score"]:
        best_store[key]["score"] = float(score)
        best_store[key]["payload"] = payload

def save_pickle(obj, path: Path):
    with open(path, "wb") as f:
        pickle.dump(obj, f)

def save_meta(meta: dict, path: Path):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)

def print_winners():
    for (ds_name, mtype, ttype), info in best_store.items():
        if info["payload"] is None:
            continue
        p = info["payload"]
        sc = p.get("score", {})
        cv = sc.get("C_v", np.nan)
        um = sc.get("U_mass", np.nan)
        print(f"🏆 {ds_name} | {mtype} | {ttype}: K={p.get('k')} | C_v={cv:.3f} | U_mass={um:.3f}")


In [11]:
def train_neutral_bigram_model(datasets, min_count=10, threshold=15):
    print("🔗 מאמן מודל Bigrams נייטרלי (clean + lemma)...")
    all_sentences = []
    for _, df in datasets.items():
        all_sentences.extend([doc.split() for doc in df['clean_text'].tolist()])
        all_sentences.extend([doc.split() for doc in df['lemma_text'].tolist()])
    phrase_model = Phrases(all_sentences, min_count=min_count, threshold=threshold)
    return Phraser(phrase_model)

bigram_phraser = train_neutral_bigram_model(datasets, min_count=10, threshold=15)


🔗 מאמן מודל Bigrams נייטרלי (clean + lemma)...


In [12]:
def build_gensim_artifacts(df, txt_col, bigram_phraser, no_below=5, no_above=0.5):
    """
    מחזיר:
    tokens: List[List[str]]
    dictionary: gensim Dictionary
    corpus_bow: List[List[(id, count)]]
    corpus_tfidf: TF-IDF view על corpus_bow
    """
    texts = df[txt_col].tolist()
    tokens = [bigram_phraser[t.split()] for t in texts]

    dictionary = corpora.Dictionary(tokens)
    dictionary.filter_extremes(no_below=no_below, no_above=no_above)

    corpus_bow = [dictionary.doc2bow(t) for t in tokens]

    tfidf_model = TfidfModel(corpus_bow)
    corpus_tfidf = tfidf_model[corpus_bow]

    return tokens, dictionary, corpus_bow, corpus_tfidf


def coherence_scores(model, tokens, dictionary, corpus_bow):
    """
    מחזיר (C_v, U_mass)
    """
    cv = CoherenceModel(model=model, texts=tokens, dictionary=dictionary, coherence='c_v').get_coherence()
    um = CoherenceModel(model=model, corpus=corpus_bow, dictionary=dictionary, coherence='u_mass').get_coherence()
    return cv, um


def top_topics_gensim(model, n_topics=5, n_words=10):
    for tid, topic in model.print_topics(num_topics=n_topics, num_words=n_words):
        clean = topic.replace('"', '').replace(' + ', ', ')
        print(f"Topic {tid}: {clean}")


In [13]:
import numpy as np
import pandas as pd
from top2vec import Top2Vec
from sentence_transformers import SentenceTransformer
import torch
from pathlib import Path

# --- הגדרות persistency ל-Kaggle ---
KAGGLE_OUT = MODELS_DIR / "top2vec_comparison"
KAGGLE_OUT.mkdir(parents=True, exist_ok=True)

# 1. טעינת המודל הניורוני (AlephBERT) - נטען פעם אחת לזיכרון ה-GPU
EMBED_HF = "imvladikon/sentence-transformers-alephbert"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🧠 Loading AlephBERT on {device}...")
st_model = SentenceTransformer(EMBED_HF, device=device)

def alephbert_embed(texts):
    # ביטול ה-Progress Bar למניעת הצפת לוגים (batch_size=64 לניצול GPU)
    return st_model.encode(texts, batch_size=64, show_progress_bar=False, convert_to_numpy=True)

# 2. פונקציית האימון המנצחת
def train_top2vec_model(docs, ds_name, txt_type):
    model_name = f"{ds_name}_{txt_type}"
    print(f"\n🚀 Starting Training: [{model_name}] | Docs: {len(docs)}")

    # פרמטרים ברזולוציה גבוהה (High-Resolution) למניעת קריסה לטופיק 0
    umap_args = {
        "n_neighbors": 15,
        "n_components": 5,
        "metric": "cosine",
        "random_state": 42
    }

    hdbscan_args = {
        "min_cluster_size": 15,
        "min_samples": 5,
        "cluster_selection_method": "eom"
    }

    model = Top2Vec(
        documents=docs,
        embedding_model=alephbert_embed,
        umap_args=umap_args,
        hdbscan_args=hdbscan_args,
        tokenizer=lambda doc: [t for t in doc.split() if len(t) >= 2],
        speed="deep-learn",
        workers=4,
        verbose=True
    )

    # שמירה פיזית ב-Kaggle
    save_path = KAGGLE_OUT / f"T2V_{model_name}"
    model.save(str(save_path))
    print(f"✅ Model [{model_name}] saved to: {save_path}")
    return model

# 3. הרצה שיטתית (Grid Run) על כל הקומבינציות
all_t2v_results = {}

for ds_name in ['Wiki', 'UGC']:
    for txt_type in ['clean_text', 'lemma_text']:
        # שליפת הטקסט הרלוונטי
        docs = datasets[ds_name][txt_type].fillna("").tolist()

        # אימון ושמירה
        model_key = f"{ds_name}_{txt_type}"
        all_t2v_results[model_key] = train_top2vec_model(docs, ds_name, txt_type)

print("\n" + "="*30)
print("🏆 All 4 Top2Vec models trained and saved!")
print("="*30)

🧠 Loading AlephBERT on cuda...


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

✅ Model [Wiki_lemma_text] saved to: /kaggle/working/HebrewTopicModel/models/top2vec_comparison/T2V_Wiki_lemma_text

🚀 Starting Training: [UGC_clean_text] | Docs: 5098


2026-01-21 21:40:21,027 - top2vec - INFO - Creating joint document/word embedding
INFO:top2vec:Creating joint document/word embedding
2026-01-21 21:40:55,236 - top2vec - INFO - Creating lower dimension embedding of documents
INFO:top2vec:Creating lower dimension embedding of documents
2026-01-21 21:41:07,278 - top2vec - INFO - Finding dense areas of documents
INFO:top2vec:Finding dense areas of documents
2026-01-21 21:41:07,486 - top2vec - INFO - Finding topics
INFO:top2vec:Finding topics
2026-01-21 21:41:07,638 - top2vec - INFO - Pre-processing documents for training
INFO:top2vec:Pre-processing documents for training
2026-01-21 21:41:07,714 - top2vec - INFO - Creating joint document/word embedding
INFO:top2vec:Creating joint document/word embedding


✅ Model [UGC_clean_text] saved to: /kaggle/working/HebrewTopicModel/models/top2vec_comparison/T2V_UGC_clean_text

🚀 Starting Training: [UGC_lemma_text] | Docs: 5098


2026-01-21 21:41:25,540 - top2vec - INFO - Creating lower dimension embedding of documents
INFO:top2vec:Creating lower dimension embedding of documents
2026-01-21 21:41:37,035 - top2vec - INFO - Finding dense areas of documents
INFO:top2vec:Finding dense areas of documents
2026-01-21 21:41:37,435 - top2vec - INFO - Finding topics
INFO:top2vec:Finding topics


✅ Model [UGC_lemma_text] saved to: /kaggle/working/HebrewTopicModel/models/top2vec_comparison/T2V_UGC_lemma_text

🏆 All 4 Top2Vec models trained and saved!


In [14]:
# =========================
# EXPORT — Stage3 Top2Vec artifacts
# =========================
from pathlib import Path
import numpy as np
import pandas as pd
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel

assert "all_t2v_results" in globals() and len(all_t2v_results) > 0
assert "datasets" in globals()

RESULTS_ROOT = RESULTS_DIR
RESULTS_ROOT.mkdir(parents=True, exist_ok=True)

TOP_WORDS = 12
TOP_DOCS  = 3

metrics_rows, topics_rows, examples_rows = [], [], []

for model_key, t2v in all_t2v_results.items():
    # model_key נראה אצלך כמו: Wiki_clean_text / UGC_lemma_text וכו'
    # נפרק ל-dataset + type
    parts = model_key.split("_")
    ds_name = parts[0]
    txt_type = "_".join(parts[1:])

    dataset_name = f"{ds_name}:{txt_type}"
    docs = datasets[ds_name][txt_type].fillna("").tolist()

    # topics
    topic_words, word_scores, topic_nums = t2v.get_topics()
    topic_sizes, topic_nums2 = t2v.get_topic_sizes()

    # diversity
    flat = [w for t in topic_words for w in t[:TOP_WORDS]]
    diversity = len(set(flat)) / len(flat) if len(flat) else np.nan

    # coherence c_v
    tokenized = [d.split() for d in docs]
    dictionary = Dictionary(tokenized)
    cm = CoherenceModel(topics=[list(t[:TOP_WORDS]) for t in topic_words], texts=tokenized, dictionary=dictionary, coherence="c_v")
    coherence_cv = cm.get_coherence()

    metrics_rows.append({
        "stage": 3,
        "platform": "kaggle",
        "model": "Top2Vec",
        "dataset": dataset_name,
        "num_topics": int(len(topic_nums)),
        "coherence_cv": float(coherence_cv),
        "topic_diversity": float(diversity),
        "notes": ""
    })

    for i, tnum in enumerate(topic_nums):
        topics_rows.append({
            "model": "Top2Vec",
            "dataset": dataset_name,
            "topic_id": int(tnum),
            "top_terms": ", ".join(topic_words[i][:TOP_WORDS]),
            "topic_size": int(topic_sizes[i]) if i < len(topic_sizes) else np.nan
        })

        # examples
        try:
            ex_docs, ex_scores, ex_ids = t2v.search_documents_by_topic(topic_num=int(tnum), num_docs=TOP_DOCS)
            for d, did in zip(ex_docs, ex_ids):
                examples_rows.append({
                    "model": "Top2Vec",
                    "dataset": dataset_name,
                    "topic_id": int(tnum),
                    "doc_id": int(did),
                    "doc_text_short": str(d)[:400].replace("\n", " ")
                })
        except Exception:
            pass

pd.DataFrame(metrics_rows).to_csv(RESULTS_ROOT / "metrics_top2vec.csv", index=False)
pd.DataFrame(topics_rows).to_csv(RESULTS_ROOT / "topics_top2vec.csv", index=False)
pd.DataFrame(examples_rows).to_csv(RESULTS_ROOT / "examples_top2vec.csv", index=False)

print("✅ Saved:", RESULTS_ROOT.resolve())


✅ Saved: /content/results_artifacts/stage3_top2vec


## Summary
מציג את שורות התוצאות של Top2Vec מתוך `all_results` (לנוחות).

**תוצרים שמורים:** `metrics_top2vec.csv`, `topics_top2vec.csv`, `examples_top2vec.csv` ב־`results_artifacts/`.


In [15]:
import pandas as pd
pd.DataFrame([r for r in all_results if r.get('Model')=='Top2Vec'])


""


In [16]:
!pip install top2vec[sentence_transformers]
!pip install top2vec-sentence-transformer # במקרים מסוימים נדרש גם זה
!pip install sentence-transformers

ERROR: Could not find a version that satisfies the requirement top2vec-sentence-transformer (from versions: none)
ERROR: No matching distribution found for top2vec-sentence-transformer


In [17]:
import numpy as np
from sentence_transformers import SentenceTransformer
from top2vec import Top2Vec

# docs כבר קיים אצלך (List[str])
# docs = ...

st = SentenceTransformer("imvladikon/sentence-transformers-alephbert")

def embed(texts):
    return st.encode(
        texts,
        convert_to_numpy=True,
        show_progress_bar=False,
        normalize_embeddings=True,
        batch_size=32
    ).astype(np.float32)

# אופציונלי: סינון מסמכים קצרים (אם אתה לא רוצה — תמחק את השורה הזאת)
docs = [d for d in docs if isinstance(d, str) and len(d.split()) >= 8]

model = Top2Vec(
    documents=docs,
    embedding_model=embed,   # חשוב: פונקציה
    min_count=2,             # כדי לא לקבל את השגיאה של min_count=50
    ngram_vocab=True,
    umap_args={"n_neighbors": 15, "n_components": 5, "min_dist": 0.0, "metric": "cosine", "random_state": 42},
    hdbscan_args={"min_cluster_size": 12, "min_samples": 3, "cluster_selection_method": "eom", "prediction_data": True},
    speed="learn",
    verbose=True
)

print("topics:", model.get_num_topics())
tw, _, tn = model.get_topics(min(5, model.get_num_topics()))
for i in range(len(tn)):
    print(tn[i], ":", ", ".join(tw[i][:10]))


2026-01-21 21:42:03,974 - top2vec - INFO - Pre-processing documents for training
INFO:top2vec:Pre-processing documents for training
2026-01-21 21:42:06,078 - top2vec - INFO - Creating joint document/word embedding
INFO:top2vec:Creating joint document/word embedding
2026-01-21 21:42:24,333 - top2vec - INFO - Creating lower dimension embedding of documents
INFO:top2vec:Creating lower dimension embedding of documents
2026-01-21 21:42:42,566 - top2vec - INFO - Finding dense areas of documents
INFO:top2vec:Finding dense areas of documents
2026-01-21 21:42:42,918 - top2vec - INFO - Finding topics
INFO:top2vec:Finding topics


topics: 3
0 : הרס כלכלה, נכנס בידוד, הכניס בידוד, פושע כלא, לקח חשבון, שנה שלטון, נמצא בידוד, מיליון מובטל, יצא רחוב, קיבל שכר
1 : קבוצה דלק, הרס כלכלה, לקח חשבון, קבוצה סיכון, מכבי חיפה, קיבל שכר, כסף ביטוח, רופא אחות, חזרה עבודה, פושע כלא
2 : יחיד קצבה, ותיק קצבה, מזכה קצבה, קרן ריבית, אמנה אזרח, קיבל משכורת, קצבה זקנה, העלאה קצבה, עובד זכאי, עצמאי משלם


In [18]:
for ds_name, df in datasets.items():
    print("\n" + "="*20)
    print(f"📚 Dataset: {ds_name}")


📚 Dataset: Wiki

📚 Dataset: UGC
